# Multilayer Perceptron with Tensorflow to detect fertility issues

In this notebook I show an implementation of a Multilayer Perceptron using tensorflow to detect fertility issues based on the next features:

* Season in which the analysis was performed. 1) winter, 2) spring, 3) Summer, 4) fall. (-1, -0.33, 0.33, 1) 
* Age at the time of analysis. 18-36 (0, 1) 
* Childish diseases (ie , chicken pox, measles, mumps, polio)	1) yes, 2) no. (0, 1) 
* Accident or serious trauma 1) yes, 2) no. (0, 1) 
* Surgical intervention 1) yes, 2) no. (0, 1) 
* High fevers in the last year 1) less than three months ago, 2) more than three months ago, 3) no. (-1, 0, 1) 
* Frequency of alcohol consumption 1) several times a day, 2) every day, 3) several times a week, 4) once a week, 5) hardly ever or never (0, 1) 
* Smoking habit 1) never, 2) occasional 3) daily. (-1, 0, 1) 
* Number of hours spent sitting per day ene-16	(0, 1) 
* Final Class: Diagnosis	normal (N), altered (O)	

This dataset contains 10 features wich are showed above. The basic idea is determine if the diagnosis is normal or altered based on 9 features related to socio-demographic factors.
The dataset was obtainde from: https://archive.ics.uci.edu/ml/datasets/Fertility

Note: This is a simple example, the idea is to ilustrate how to implement a tensorflow model with a toy dataset.

## Getting started

In [1]:
# Import stuff
import numpy as np
import pandas as pd
import tensorflow as tf

# Load dataset
file = pd.read_csv('fertility_Diagnosis.txt', header=None)

In [2]:
# Show part of the entery dataset.
# Check column 9. Values are 'N' or 'O' wich mean normal or alterated. We have to change them to values that
# we can work with. 
file[0:10]

,0,1,2,3,4,5,6,7,8,9
0,-0.33,0.69,0,1,1,0,0.8,0,0.88,N
1,-0.33,0.94,1,0,1,0,0.8,1,0.31,O
2,-0.33,0.50,1,0,0,0,1.0,-1,0.50,N
3,-0.33,0.75,0,1,1,0,1.0,-1,0.38,N
4,-0.33,0.67,1,1,0,0,0.8,-1,0.50,O
5,-0.33,0.67,1,0,1,0,0.8,0,0.50,N
6,-0.33,0.67,0,0,0,-1,0.8,-1,0.44,N
7,-0.33,1.00,1,1,1,0,0.6,-1,0.38,N
8,1.00,0.64,0,0,1,0,0.8,-1,0.25,N
9,1.00,0.61,1,0,0,0,1.0,-1,0.25,N


In [3]:
# Change values from column 9
file[9] = file[9].map({'N':0, 'O':1})

In [4]:
file[0:10]

,0,1,2,3,4,5,6,7,8,9
0,-0.33,0.69,0,1,1,0,0.8,0,0.88,0
1,-0.33,0.94,1,0,1,0,0.8,1,0.31,1
2,-0.33,0.50,1,0,0,0,1.0,-1,0.50,0
3,-0.33,0.75,0,1,1,0,1.0,-1,0.38,0
4,-0.33,0.67,1,1,0,0,0.8,-1,0.50,1
5,-0.33,0.67,1,0,1,0,0.8,0,0.50,0
6,-0.33,0.67,0,0,0,-1,0.8,-1,0.44,0
7,-0.33,1.00,1,1,1,0,0.6,-1,0.38,0
8,1.00,0.64,0,0,1,0,0.8,-1,0.25,0
9,1.00,0.61,1,0,0,0,1.0,-1,0.25,0


In [5]:
# Transform a numpy array
file = file.values


In [6]:
# Split train and test
x_train = file[0:90, 0:9]
y_train = file[0:90, 9]
x_test = file[90:100, 0:9]
y_test = file[90:100, 9]

In [7]:
# Reshape target vector
y_train = np.reshape(y_train, (y_train.shape[0], 1))
y_test = np.reshape(y_test, (y_test.shape[0], 1))

In [8]:
# Print shapes
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(90, 9)
(10, 9)
(90, 1)
(10, 1)


## Neural network hyperparameters

In [9]:
# Declare hyperparameters and number of hidden units
learning_rate = 0.1
training_epochs = 10

units_hidden_1 = 128
units_hidden_2 = 64
features_input = 9
number_clases = 1

In [10]:
# Create respective placeholders

x = tf.placeholder('float', [None, features_input])
y = tf.placeholder('float', [None, number_clases])

## The model

In [11]:
# Defining the model

def MultilayerPerceptron(x, weights, biases):
    print('x: ', x.get_shape(),
         'w1: ', weights['h1'].get_shape(),
         'b1: ', biases['b1'].get_shape())
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1']) # A1 = xw + b
    layer_1 = tf.nn.relu(layer_1) # relu(A)
    
    print('layer_1: ', layer_1.get_shape(),
         'w2: ', weights['h2'].get_shape(),
         'b2: ', biases['b2'].get_shape())
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2']) # A2 = xw + b
    layer_2 = tf.nn.relu(layer_2)
    
    print('layer_2:', layer_2.get_shape(),
         'w3: ', weights['out'].get_shape(),
         'b3: ', biases['out'].get_shape())
    output_layer = tf.matmul(layer_2, weights['out']) + biases['out'] # out = xw + b
    print('output_layer: ', output_layer.get_shape())
    
    return output_layer

## Initialize weights

In [12]:
# Defining weights and biases
weights = {
    'h1': tf.Variable(tf.random_normal([features_input, units_hidden_1])),
    'h2': tf.Variable(tf.random_normal([units_hidden_1, units_hidden_2])),
    'out': tf.Variable(tf.random_normal([units_hidden_2, number_clases]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([units_hidden_1])),
    'b2': tf.Variable(tf.random_normal([units_hidden_2])),
    'out': tf.Variable(tf.random_normal([number_clases]))
}

pred = MultilayerPerceptron(x, weights, biases)

x:  (?, 9) w1:  (9, 128) b1:  (128,)
layer_1:  (?, 128) w2:  (128, 64) b2:  (64,)
layer_2: (?, 64) w3:  (64, 1) b3:  (1,)
output_layer:  (?, 1)


## Loss and optimizer

In [13]:
# Defining costo function and optimizer
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

## Launch graph

In [14]:
# Lauching the graph
init = tf.initialize_all_variables()

# Initializing session
with tf.Session() as sess:
    sess.run(init)
    
    # Running every epoch
    for epoch in range(training_epochs):
        avg_cost = 0
        _, c = sess.run([optimizer, cost], feed_dict={x : x_train, y: y_train})
        avg_cost += c / 90
        print('Epoch: ', '%04d' % (epoch+1), 'Cost: ', '{:.9f}'.format(avg_cost))
       
    # Calculate prediction
    correct_prediction = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
    
    print('Accuracy: ', accuracy.eval({x:x_test, y:y_test}))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch:  0001 Cost:  0.140563541
Epoch:  0002 Cost:  0.139132341
Epoch:  0003 Cost:  0.812749651
Epoch:  0004 Cost:  0.205992381
Epoch:  0005 Cost:  0.385366228
Epoch:  0006 Cost:  0.425470267
Epoch:  0007 Cost:  0.379024929
Epoch:  0008 Cost:  0.295779461
Epoch:  0009 Cost:  0.204935943
Epoch:  0010 Cost:  0.119415453
Accuracy:  1.0


## Conclusion

It have showed how to implement a tensorflow model in a toy dataset making use of a multilayer perceptron. 